# Multi Agent Systems

A team of group of agents that work together to achieve a common goal.

In [10]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENROUTER_API_KEY")

model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model = "openai/gpt-oss-20b:free",
    api_key=api_key,
    model_info={
        "family":"openai",
        "vision" : True,
        "function_calling":True,
        "json_output":False,
    }
)

x:\Autogen\autogen-env\Lib\site-packages\autogen_ext\models\openai\_openai_client.py:453: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


# Single Agent Approach

single agent is going to create a short story for us

In [11]:
story_agent = AssistantAgent(
    name="story_agent",
    model_client=model_client,
    system_message="You are a creating writer. Generate a short story about a brave knight and a dragon"
)

In [12]:
from autogen_agentchat.messages import TextMessage

async def test_simple_agent():
    task = TextMessage(
        content = "Write a short story about a brave knight and a dragon. keep it upto 200 words",
        source = "user"
    )
    result = await story_agent.run(task=task)
    print(result.messages[-1].content)
    
await test_simple_agent()

Sir Elric of Greycrest rode into the mist‑cloaked valley with a single, silver lance and a heart that missed no fear. The dragon, Amarath, lay coiled beneath the ancient oak, scales glittering like broken glass in the moonlight. The village’s cries of fire had reached Elric’s ears, but he knew the beast was not merely a monster; it was a guardian of a forgotten covenant.

 “Stand down!” Elric shouted, his voice cutting the wind.  
Amarath’s eyes opened, amber and ancient. “Why do you disobey the pact?” the dragon breathed, its voice echoing through the trees.

Elric lowered his lance slightly. “Because I see terror in the villagers’ eyes, not the curse you protect. I seek not to die, but to free them.”

The dragon’s wings flickered. “I was bound to keep this valley warm with fire, until a human shone hope within my heart. That is you, Sir Knight.”

With a gentle squeeze of the lance’s tip, Elric touched the dragon’s snout. A ripple of light surged over Amarath; fire turned to cooling m

# Multi Agent Team Approach

We will be using 3 agents to now write the story, each with its own work and mastery.

In [13]:
plot_agent = AssistantAgent(
    name ='plot_agent',
    model_client=model_client,
    system_message="you create engaging plots for stories. Focus on the knight's journey"
)

character_agent = AssistantAgent(
    name ="character_agent",
    model_client=model_client,
    system_message="You develop Characters. Describe the knight and the Dragon in details, including their motivations and backgrounds."
)

ending_agent = AssistantAgent(
    name="ending_writer",
    model_client=model_client,
    system_message="You write engaging endings. Conclude the story with a twist."
)    

In [14]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [ plot_agent, character_agent, ending_agent],
    max_turns= 3,
)

async def test_team():
    task = TextMessage(
        content = "Write a short story about a brave knight and a dragon. keep it upto 400 words",
        source = "user")
    result = await team.run(task=task)
    for each_agent_messg in result.messages:
        print(f'{each_agent_messg.source}:{each_agent_messg.content}')

await test_team()

user:Write a short story about a brave knight and a dragon. keep it upto 400 words
plot_agent:Sir Rowan rode from the mist‑shrouded Vale with a horn pressing against his cuirass. The villagers had whispered for months that a fearsome dragon lay atop the Eastern Spine, breathing fire that turned hearth‑wood to ash. Rowan’s sister, a midwife, had slipped a wisp of her health into his mind. “No dragon can win against the will of a heart that heals,” she’d said, waving his sword as she elbowed a bundle of herbs around the knob of his breastplate.

He crossed stone arches, slipped into rock‑taken ridges, and, as twilight kissed the horizon, he finally saw the creature. It was no tyrant of lore—scales mottled like summer moss, eyes that glowed not with malice but with a weary loneliness. The dragon’s cry was not a threat; it was a lament, the echo of distant glades that once sang with dragonsong.

“Who dares approach my dwelling?” the dragon intoned, its voice rumbling through the canyon.

R

# Comparison Agent

Aspect:

1. Creativity:

Single Agent:  Limited to one viewpoint whereas multiagent team - Diverse ideas from each agent.

2. Depth:

Single agent: Basic story elements  
multi agent : Detailed plot, Characters and endings

3. Flexibility:

Single agent : stuck to one stype
multi agent : adaptable with specialized roles and styles

4. Setup effort:

single agent : simple setup
multi agent: more agents and teams setup which can provide better output.